## Fuse ETAS & Darknet offline logging

#### ETAS Log

* psipunkt - yaw_rate
* psi - yaw
* delta - steering angle

```
timestamp,filename,ax[m/s^2],ay[m/s^2],vx[m/s],vy[m/s],psipunkt[r],psi[r],delta[r],timetoread[µs]
```
```
1527877716648,/home/nvidia/ssd/2018_06_04_Jesenwang/0/1527877716638042.bmp,0.138672,-0.175781,0,0,-0.000915528,0.690291,0.0368155,269
```

We extract the filename, timestamp, vx, vy, yaw.

Remove first line, delimiter are '\n'.

#### Darknet Log

* filepath
* distance (meter)
* class [0 - yellow, 1 - blue, 2 - red]
* angle (rad) - (vehicle coord. system)

```
#
/media/fsd/TOSHIBA EXT/jesenwang-testtag_2018-06-04/2018_06_04_Jesenwang/straight/1/1527877830504357.jpg
2.84854, 2, 0.582889
2.69438, 2, -0.489339
#
/media/fsd/...
```

We extract the filename a counter starting from 0 for each image and a list of [distance, angle, class]

Remove first 10 lines, delimiter are '#'.

## Result

We want a logfile with the following format:
```
distance, angle, x_car, y_car, yaw, v_x, v_y, color, counter, timestamp
```

* `distance` (meter) - to object
* `angle` (rad) - to object
* `x_car` - set to 0, we don't have ground truth
* `y_car` - set to 0, we don't have ground truth
* `yaw` - yaw of the car
* `v_x` - velocity car (longitudinal)
* `v_y` - velocity car (lateral)
* `color` - (0: yellow, 1: blue, 2: red)
* `counter` - increment for each image, start from 0
* `timestamp` - timestamp in ms


In [6]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()
import matplotlib.pyplot as plt
import numpy             as np
import csv

### Darknet Logging

In [91]:
def get_filename(line):
    '''
    /path/to/my/image/image000001.bmp to image000001
    '''
    filename_ext = line.split('/')[-1]
    return filename_ext.split('.')[0]

def get_filename_from_block(string_block):
    '''
        0 #
        1 /path/to/my/image/image000001.bmp
        2 distance, angle, color
        3 distance, angle, color
        4
        
        Extracts the filename
    '''
    line = string_block.split('\n')[1]
    return get_filename(line)

    
def parse_darknet_log(path):
    '''
        0 # 
        1 /path/to/my/image/image000001.bmp
        2 distance, color, angle
        3 distance, color, angle 
        4
        
        Extracts filename, distance, angle, color and counter
    '''
    
    logging = []
    
    with open(path, 'rb') as file:
        data = file.read()
        counter = 0
        for string_block in data.split('#')[1:]: # drop everything before the first #
            filename = get_filename(string_block)
            lines = string_block.split('\n')[2:] # drop 2 front lines 
            lines = lines[:-1]                   # drop last line
            for ix in range(0, len(lines)):
                values = lines[ix].split(',')
                distance, color, angle = list(map(lambda s: s.strip(), values))
                logging.append([filename, distance, angle, color, counter])
            counter = counter + 1
    return logging

### ETAS Logging

In [195]:
def parse_etas_log(path):
    '''
        0 timestamp,filename,ax[m/s^2],ay[m/s^2],vx[m/s],vy[m/s],psipunkt[r],psi[r],delta[r],timetoread[µs]
        1 1527877830512,/home/nvidia/ssd/2018_06_04_Jesenwang/1/1527877830504357.bmp,0.142578,-0.175781,0,0,-0.000305176,0.696427,0.0490874,83
        2 1527877830530, ..
    '''
    
    logging = []
    
    with open(path, 'r') as csvfile:
        reader = csv.reader(csvfile, skipinitialspace=False, delimiter=',')
        counter = 0
        for row in reader: 
            if counter == 0: # skip first line
                counter += 1
                continue
            timestamp = row[0]
            if timestamp.split()[0][0] != '1': # not a valid line
                continue
            filepath  = get_filename(row[1])
            ax = row[2]
            ay = row[3]
            vx = row[4]
            vy = row[5]
            yaw_rate = row[6]
            yaw = row[7]
            steering_angle = row[8]
            delta_time = float(row[9]) / 1000 # convert in milliseconds
            logging.append([filepath, timestamp, vx, vy, yaw, yaw_rate])
    
    old_time = int(logging[0][1])
    logging[0][4] = 0
    old_yaw = logging[0][4]
    for l in logging[1:]:
        cur_time = int(l[1])
        yaw_rate = l[5]
        time_diff = cur_time - old_time
        l[4] = float(yaw_rate) * (float(time_diff) / 1000.0)
        l[4] += old_yaw
        old_yaw = l[4]
        old_time = cur_time
    
    return logging

In [196]:
def fuse_loggings(darknet_logging, etas_logging):
    '''
    darknet - filename, distance, angle, color, counter
    etas    - filename, timestamp, vx, vy, yaw

    JOIN on filename, return 'distance, angle, x_car, y_car, yaw, v_x, v_y, color, counter, timestamp'
    '''
    
    dark_index = 0
    etas_index = 0
    logging = []

    null_yaw = float(etas_logging[etas_index][4]) # we have to null yaw (subtract the first value from all)
    
    while dark_index != (len(darknet_logging)-1):
        darknet_filename = darknet_logging[dark_index][0]
        etas_filename = etas_logging[etas_index][0]
        if darknet_filename == etas_filename:
            distance = darknet_logging[dark_index][1]
            angle = darknet_logging[dark_index][2]
            x_car = 0
            y_car = 0
            yaw = float(etas_logging[etas_index][4]) - null_yaw
            v_x = float(etas_logging[etas_index][2])
            v_y = float(etas_logging[etas_index][3])
            color = darknet_logging[dark_index][3]
            counter = darknet_logging[dark_index][4]
            timestamp = int(etas_logging[etas_index][1]) / 1000.0 # milliseconds to seconds
            
            logging.append([distance, angle, x_car, y_car, yaw, v_x, v_y, color, counter, timestamp])
            
            dark_index += 1
        else:
            etas_index += 1
    return logging

In [197]:
def write_logging(filepath):
    
    with open(filepath + '.csv', 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for line in loggings:
            writer.writerow(line)

In [198]:
darknet_logfile_path = '../example-data/car/oval/2018-06-04_jo6-large-darknet-v2.log'
etas_logfile_path = '../example-data/car/oval/2018-06-04_jo6-first_test'

parsed_darknet_logging = parse_darknet_log(darknet_logfile_path)
parsed_etas_logging = parse_etas_log(etas_logfile_path)

# print sample data
print('darknet logging ({} samples) - filename, distance, angle, color, counter'.format(len(parsed_darknet_logging)))
for e in parsed_darknet_logging[0:10]:
    print(e)
print('-----')
print('etas logging ({} samples) - filename, timestamp, vx, vy, yaw'.format(len(parsed_etas_logging)))
for e in parsed_etas_logging[0:10]:
    print(e)
print('-----')

# fuse loggings
loggings = fuse_loggings(parsed_darknet_logging, parsed_etas_logging)
print('summary logging ({} samples) - distance, angle, x_car, y_car, yaw, v_x, v_y, color, counter, timestamp'.format(len(loggings)))
for l in loggings[0:20]:
    print(l)

output_filepath = '../example-data/2018-06-04-jesenwang-oval-06-fused-logging'
write_logging(output_filepath)
    

darknet logging (19193 samples) - filename, distance, angle, color, counter
['1527880922023409', '8.12264', '0.245697', '3', 0]
['1527880922023409', '8.39846', '-0.105886', '3', 0]
['1527880922023409', '3.96003', '0.410181', '1', 0]
['1527880922039217', '8.39846', '-0.105886', '3', 1]
['1527880922039217', '8.12264', '0.245697', '3', 1]
['1527880922039217', '3.96003', '0.410181', '1', 1]
['1527880922053225', '8.39846', '-0.105886', '3', 2]
['1527880922053225', '8.12264', '0.245697', '3', 2]
['1527880922053225', '3.9565', '0.408125', '1', 2]
['1527880922067419', '8.12264', '0.245697', '3', 3]
-----
etas logging (9723 samples) - filename, timestamp, vx, vy, yaw
['1527880922023409', '1527880922030', '1.87988', '0.012207', 0, '0.0549317']
['1527880922039217', '1527880922045', '1.83105', '0', 0.0007781985, '0.0518799']
['1527880922053225', '1527880922059', '1.78223', '-0.012207', 0.0014532477, '0.0482178']
['1527880922067419', '1527880922074', '1.75781', '-0.0244141', 0.0020437631999999997, 